In [1]:
"""
Assignment 3 - clustering
"""

import sys
import pdb
sys.path.append("C:\\users\\mccar\\miniconda3\\lib\\site-packages")
import numpy as np
import pandas as pd
import os
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.metrics import accuracy_score, silhouette_score, mutual_info_score
from sklearn.model_selection import validation_curve, GridSearchCV, StratifiedKFold, train_test_split
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture
from sklearn.decomposition import PCA, FastICA
from sklearn.random_projection import GaussianRandomProjection
from scipy.stats import kurtosis


In [2]:
# pd.set_option('display.max_columns', 60)
train_df = pd.read_csv("data/gtzan/features_3_sec.csv")
train_df.head()
# print(train_df.shape)
# print(train_df.info())

,filename,length,chroma_stft_mean,chroma_stft_var,rms_mean,rms_var,spectral_centroid_mean,spectral_centroid_var,spectral_bandwidth_mean,spectral_bandwidth_var,...,mfcc16_var,mfcc17_mean,mfcc17_var,mfcc18_mean,mfcc18_var,mfcc19_mean,mfcc19_var,mfcc20_mean,mfcc20_var,label
0,blues.00000.0.wav,66149,0.335406,0.091048,0.130405,0.003521,1773.065032,167541.630869,1972.744388,117335.771563,...,39.687145,-3.241280,36.488243,0.722209,38.099152,-5.050335,33.618073,-0.243027,43.771767,blues
1,blues.00000.1.wav,66149,0.343065,0.086147,0.112699,0.001450,1816.693777,90525.690866,2010.051501,65671.875673,...,64.748276,-6.055294,40.677654,0.159015,51.264091,-2.837699,97.030830,5.784063,59.943081,blues
2,blues.00000.2.wav,66149,0.346815,0.092243,0.132003,0.004620,1788.539719,111407.437613,2084.565132,75124.921716,...,67.336563,-1.768610,28.348579,2.378768,45.717648,-1.938424,53.050835,2.517375,33.105122,blues
3,blues.00000.3.wav,66149,0.363639,0.086856,0.132565,0.002448,1655.289045,111952.284517,1960.039988,82913.639269,...,47.739452,-3.841155,28.337118,1.218588,34.770935,-3.580352,50.836224,3.630866,32.023678,blues
4,blues.00000.4.wav,66149,0.335579,0.088129,0.143289,0.001701,1630.656199,79667.267654,1948.503884,60204.020268,...,30.336359,0.664582,45.880913,1.689446,51.363583,-3.392489,26.738789,0.536961,29.146694,blues


In [3]:
# Preprocessing
train_df = train_df.drop(['filename', 'length'], axis = 1)
y = train_df['label']
X = train_df.drop('label', axis = 1)
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [4]:
# More preprocessing
scale = MinMaxScaler()
scaled_data = scale.fit_transform(x_train)
x_train_sc = pd.DataFrame(scaled_data, columns = x_train.columns).values
scaled_data = scale.fit_transform(x_test)
x_test_sc = pd.DataFrame(scaled_data, columns = x_test.columns).values
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y_train)

In [5]:
# K Means

run = True
if run:
    # Define the range of cluster numbers (K)
    k_range = range(2, 15)

    # Plotting the clustering results for varying numbers of clusters
    silhouette_scores = []
    for k in k_range:
        kmeans = KMeans(n_clusters=k)
        kmeans.fit(x_train_sc)
        labels = kmeans.labels_
        score = silhouette_score(x_train_sc, labels)
        silhouette_scores.append(score)

    # Plot the silhouette scores vs. K
    plt.plot(k_range, silhouette_scores, marker='o')
    plt.title('KMeans GTZAN - Silhouette Score vs. Number of Clusters (K)')
    plt.xlabel('Number of Clusters (K)')
    plt.ylabel('Silhouette Score')
    plt.xticks(k_range)
    plt.grid(True)
    plt.show()

In [6]:
# Expectation Maximation (EM) Clustering

run = False
if run:
    # Define the range of cluster numbers (K)
    k_range = range(2, 15)

    # Calculate silhouette scores for different values of K
    silhouette_scores = []
    for k in k_range:
        gmm = GaussianMixture(n_components=k)
        gmm.fit(x_train_sc)
        labels = gmm.predict(x_train_sc)
        silhouette_avg = silhouette_score(x_train_sc, labels)
        silhouette_scores.append(silhouette_avg)

    # Plot the silhouette scores vs. K
    plt.plot(k_range, silhouette_scores, marker='o')
    plt.title('Silhouette Score vs. Number of Clusters (K) EM')
    plt.xlabel('Number of Clusters (K)')
    plt.ylabel('Silhouette Score')
    plt.xticks(k_range)
    plt.grid(True)
    plt.show()

In [20]:
# PCA

# Fit PCA to the data
run = False
if run:
    pca = PCA()
    pca.fit(x_train_sc)
    x_pca = pca.fit_transform(x_train_sc)

    # Get the explained variance (eigenvalues) for each principal component
    eigenvalues = pca.explained_variance_ratio_
    cumulative_explained_variance = np.cumsum(eigenvalues)
    print(eigenvalues)
    print(sum(eigenvalues))
    print(len(eigenvalues))

    # Plotting
    fig, ax1 = plt.subplots()

    # Bar plot for explained variance
    ax1.bar(np.arange(len(eigenvalues)), eigenvalues, color='b', alpha=0.5)
    ax1.set_xlabel('Principal Components')
    ax1.set_ylabel('Explained Variance Ratio', color='b')

    # Line plot for cumulative explained variance
    ax2 = ax1.twinx()
    ax2.plot(np.arange(len(cumulative_explained_variance)), cumulative_explained_variance, color='r')
    ax2.set_ylabel('Cumulative Explained Variance Ratio', color='r')

    plt.title('PCA Analysis')
    plt.show()

    # Plot the original clusters with colors
    plt.figure(figsize=(10, 6))
    plt.scatter(x_pca[:, 0], x_pca[:, 1], c=y_encoded, cmap='viridis', alpha=0.5)
    plt.xlabel('Principal Component 1')
    plt.ylabel('Principal Component 2')
    plt.title('PCA Components with Colored Clusters')
    plt.colorbar(label='Cluster')
    plt.grid(True)
    plt.show()

    # Plot the original clusters with colors in 3D
    fig = plt.figure(figsize=(10, 8))
    ax = fig.add_subplot(111, projection='3d')
    ax.scatter(x_pca[:, 0], x_pca[:, 1], x_pca[:, 2], c=y_encoded, cmap='viridis', alpha=0.5)
    ax.set_xlabel('Principal Component 1')
    ax.set_ylabel('Principal Component 2')
    ax.set_zlabel('Principal Component 3')
    ax.set_title('PCA Components with Colored Clusters')
    plt.show()

In [22]:
# ICA
run = False
if run:
    # Fit ICA to the data
    ica = FastICA(n_components=2)
    x_ica = ica.fit_transform(x_train_sc)
    
    # Get the independent components
    components = ica.components_

    # Plot the original clusters with colors
    plt.figure(figsize=(10, 6))
    plt.scatter(x_ica[:, 0], x_ica[:, 1], c=y_encoded, cmap='viridis', alpha=0.5)
    plt.xlabel('Independent Component 1')
    plt.ylabel('Independent Component 2')
    plt.title('ICA Components with Colored Clusters')
    plt.colorbar(label='Cluster')
    plt.grid(True)
    plt.show()

    # Fit ICA to the data
    ica = FastICA(n_components=3)
    x_ica = ica.fit_transform(x_train_sc)
    
    # Get the independent components
    components = ica.components_
    # Plot the original clusters with colors in 3D
    fig = plt.figure(figsize=(10, 8))
    ax = fig.add_subplot(111, projection='3d')
    ax.scatter(x_ica[:, 0], x_ica[:, 1], x_ica[:, 2], c=y_encoded, cmap='viridis', alpha=0.5)
    ax.set_xlabel('Independent Component 1')
    ax.set_ylabel('Independent Component 2')
    ax.set_zlabel('Independent Component 3')
    ax.set_title('ICA Components with Colored Clusters')
    plt.show()


    def negentropy(x):
        return -np.sum(np.log(np.cosh(x))) - np.sum(np.log(np.sqrt(2 * np.pi * np.exp(1))))

    # Compare before and after transformation
    # 1. Mutual Information (for original data)
    mutual_info_matrix_original = np.zeros((x_train_sc.shape[1], x_train_sc.shape[1]))
    for i in range(x_train_sc.shape[1]):
        for j in range(x_train_sc.shape[1]):
            mutual_info_matrix_original[i, j] = mutual_info_score(x_train_sc[:, i], x_train_sc[:, j])

    # 2. Kurtosis (for original data)
    feature_kurtosis_original = kurtosis(x_train_sc)

    # 3. Calculate negentropy for each feature in the original data
    negentropy_original_values = np.apply_along_axis(negentropy, axis=0, arr=x_train_sc)

    # Apply Independent Component Analysis (ICA)
    ica = FastICA(random_state=42)
    ica.fit(x_train_sc)
    # Transform the original data using ICA
    ica_components = ica.transform(x_train_sc)

    # 3. Measures of Non-Gaussianity (Negentropy using ICA)
    # Calculate negentropy for each independent component obtained through ICA
    negentropy_values_transformed = np.apply_along_axis(negentropy, axis=0, arr=ica_components)

    # 4. Mutual Information (for transformed data)
    mutual_info_matrix_transformed = np.zeros((ica_components.shape[1], ica_components.shape[1]))
    for i in range(ica_components.shape[1]):
        for j in range(ica_components.shape[1]):
            mutual_info_matrix_transformed[i, j] = mutual_info_score(ica_components[:, i], ica_components[:, j])

    # 5. Kurtosis (for transformed data)
    component_kurtosis_transformed = kurtosis(ica_components)

In [23]:
print("Before Transformation:")
print("Mean Mutual Information:  {}".format(np.mean(mutual_info_matrix_original)))
print("Mean Kurtosis:           {}".format(np.mean(feature_kurtosis_original)))
print("Mean Negentropy:         {}".format(np.mean(negentropy_original_values)))
 
print("After Transformation:")
print("Mean Mutual Information:  {}".format(np.mean(mutual_info_matrix_transformed)))
print("Mean Kurtosis:           {}".format(np.mean(component_kurtosis_transformed)))
print("Mean Negentropy:         {}".format(np.mean(negentropy_values_transformed)))

Before Transformation:
Mean Mutual Information:  6.87168913319125
Mean Kurtosis:           11.541381511119251
Mean Negentropy:         -497.2155613382608
After Transformation:
Mean Mutual Information:  8.97103683364995
Mean Kurtosis:           23.332335183191415
Mean Negentropy:         -1.9186670878145102


In [12]:
# RCA

run = True
if run:
    x_rp = GaussianRandomProjection.fit_transform(x_train_sc)
    pca = PCA()
    pca.fit(x_train_sc)
    x_pca = pca.fit_transform(x_train_sc)

    # Get the explained variance (eigenvalues) for each principal component
    eigenvalues = pca.explained_variance_ratio_
    cumulative_explained_variance = np.cumsum(eigenvalues)
    print(eigenvalues)
    print(sum(eigenvalues))
    print(len(eigenvalues))

    # Plotting
    fig, ax1 = plt.subplots()

    # Bar plot for explained variance
    ax1.bar(np.arange(len(eigenvalues)), eigenvalues, color='b', alpha=0.5)
    ax1.set_xlabel('Principal Components')
    ax1.set_ylabel('Explained Variance Ratio', color='b')

    # Line plot for cumulative explained variance
    ax2 = ax1.twinx()
    ax2.plot(np.arange(len(cumulative_explained_variance)), cumulative_explained_variance, color='r')
    ax2.set_ylabel('Cumulative Explained Variance Ratio', color='r')

    plt.title('PCA Analysis')
    plt.show()

    # Plot the original clusters with colors
    plt.figure(figsize=(10, 6))
    plt.scatter(x_pca[:, 0], x_pca[:, 1], c=y_encoded, cmap='viridis', alpha=0.5)
    plt.xlabel('Principal Component 1')
    plt.ylabel('Principal Component 2')
    plt.title('PCA Components with Colored Clusters')
    plt.colorbar(label='Cluster')
    plt.grid(True)
    plt.show()

    # Plot the original clusters with colors in 3D
    fig = plt.figure(figsize=(10, 8))
    ax = fig.add_subplot(111, projection='3d')
    ax.scatter(x_pca[:, 0], x_pca[:, 1], x_pca[:, 2], c=y_encoded, cmap='viridis', alpha=0.5)
    ax.set_xlabel('Principal Component 1')
    ax.set_ylabel('Principal Component 2')
    ax.set_zlabel('Principal Component 3')
    ax.set_title('PCA Components with Colored Clusters')
    plt.show()

# TODO how to better visualize this?
run = False
if run:
    # Apply Random Projection to reduce the dimensionality
    rp = GaussianRandomProjection(n_components=4)
    X_rp = rp.fit_transform(x_train_sc)

    # Plot the reduced dimensionality data
    plt.figure(figsize=(8, 6))
    plt.scatter(X_rp[:, 0], X_rp[:, 1], alpha=0.8)
    plt.xlabel('Random Projection Dimension 1')
    plt.ylabel('Random Projection Dimension 2')
    plt.title('Random Projection')
    plt.grid(True)
    plt.show()